In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

 ### ML Flow

In [3]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

# access UI
# mlflow ui --backend-store-uri sqlite:///mlflow.db

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/04-deployment/02_web_service_mlflow/mlruns/1', creation_time=1740067319264, experiment_id='1', last_update_time=1740067319264, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
# create utility function for dataframe reading
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

### Import datasets

In [5]:
# january
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
# february
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')
len(df_train), len(df_val)

(54373, 51497)

In [6]:
# combine Pick Up Location and Drop Off Location in one feature
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()
# fit transform
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
# transform
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

X_train, X_val

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 108746 stored elements and shape (54373, 5093)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 100591 stored elements and shape (51497, 5093)>)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Train models

#### XGBoost

In [10]:
import xgboost as xgb

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    best_params = {
        'learning_rate': 0.7796116539991661,
        'max_depth': 58,
        'min_child_weight': 5.171721905211863,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.3147724697905352,
        'reg_lambda': 0.01549469142014421,
        'seed': 42
    }
    
    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=50,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    with open ('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    # save preprocessing as artifact
    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    # model saving
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[0]	validation-rmse:5.60909
[1]	validation-rmse:5.34361
[2]	validation-rmse:5.30665
[3]	validation-rmse:5.29071
[4]	validation-rmse:5.28842
[5]	validation-rmse:5.29025
[6]	validation-rmse:5.27345
[7]	validation-rmse:5.26109
[8]	validation-rmse:5.25938
[9]	validation-rmse:5.24434
[10]	validation-rmse:5.24280
[11]	validation-rmse:5.24408
[12]	validation-rmse:5.24225
[13]	validation-rmse:5.24407
[14]	validation-rmse:5.24436
[15]	validation-rmse:5.24270
[16]	validation-rmse:5.23825
[17]	validation-rmse:5.23727
[18]	validation-rmse:5.23499
[19]	validation-rmse:5.23482
[20]	validation-rmse:5.23417
[21]	validation-rmse:5.23275
[22]	validation-rmse:5.23263
[23]	validation-rmse:5.22892
[24]	validation-rmse:5.23092
[25]	validation-rmse:5.22924
[26]	validation-rmse:5.22964
[27]	validation-rmse:5.22905
[28]	validation-rmse:5.22843
[29]	validation-rmse:5.22721
[30]	validation-rmse:5.22918
[31]	validation-rmse:5.22771
[32]	validation-rmse:5.23057
[33]	validation-rmse:5.22846
[34]	validation-rmse:5.2

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:37:17] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/02/20 16:37:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Test model import

In [11]:
from mlflow.tracking import MlflowClient

RUN_ID = '1b91ef40481d4e2bacb49373ea84d4d8'
MLFLOW_TRACKING_URI = 'http://localhost:5000'

logged_model = f'runs:/{RUN_ID}/models_mlflow'

#### Load model from model registry

In [12]:
model = mlflow.pyfunc.load_model(logged_model)

#### Load artifacts from client

In [13]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [14]:
path = client.download_artifacts(run_id=RUN_ID, path='preprocessor/preprocessor.b')

In [16]:
with open(path, 'rb') as f_in:
    dv = pickle.load(f_in)

In [17]:
dv

DictVectorizer()

### Train pipeline

In [18]:
from sklearn.pipeline import make_pipeline

In [22]:
def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [25]:
dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

#### Random forest

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="models_mlflow")

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 5.440123385929052


2025/02/20 16:54:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
